# Exercise 4.7

Write a program for policy iteration and re-solve Jack’s car rental problem with the following changes.

One of Jack’s employees at the first location rides a bus home each night and lives near the second location. 

She is happy to shuttle one car to the second location for free. 

Each additional car still costs $2, as do all cars moved in the other direction. 

In addition, Jack has limited parking space at each location. 

If more than 10 cars are kept overnight at a location (after any moving of cars), then an additional cost of $4 must be incurred to use a second parking lot (independent of how many cars are kept there). 

These sorts of nonlinearities and arbitrary dynamics often occur in real problems and cannot easily be handled by optimization methods other than dynamic programming. To check your program, first replicate the results given for the original problem.

![Jack's car rental problem](ex4.2.png)

In [1]:
from dataclasses import dataclass
from typing_extensions import Self
import numpy as np

## Original Problem

In [ ]:
# Create State class
@dataclass
class State:
    cars_loc1: int
    cars_loc2: int
    
    def __post_init__(self) -> None:
        if not (0 <= self.cars_loc1 <= 20 and 0 <= self.cars_loc2 <= 20):
            raise ValueError("Cars at each location must be between 0 and 20")
        
    def move_cars(self, cars_moved_l1_to_l2: int) -> tuple[Self, int]:
        assert (-5 <= cars_moved_l1_to_l2 <= 5, "Actions can only move up to 5 cars")
        assert (cars_moved_l1_to_l2 <= self.cars_loc1 and -cars_moved_l1_to_l2 <= self.cars_loc2), "Cannot move more cars than there are at the location")
        
        reward = -2 * np.abs(cars_moved_l1_to_l2)
        
        return State(
            cars_loc1=self.cars_loc1-cars_moved_l1_to_l2, 
            cars_loc2=self.cars_loc2+cars_moved_l1_to_l2,
            ), reward
    
    @staticmethod
    def poisson_prob(n: int, lambda_: int) -> float:
        return (lambda_**n / np.math.factorial(n)) * np.exp(-lambda_)

    def rent_car(self, location: int, rented_cars: int) -> tuple[Self, float, int]:
        match location:
            case 1:
                cars_available: int = self.cars_loc1
            case 2:
                cars_available:int = self.cars_loc2
        
        assert 0 <= rented_cars <= cars_available, "Cannot rent more cars than are available"
        
        reward = 10 * rented_cars
        
        match location:
            case 1:
                return State(
                    cars_loc1=self.cars_loc1-rented_cars, 
                    cars_loc2=self.cars_loc2,
                    )
            case 2:
                return State(
                    cars_loc1=self.cars_loc1, 
                    cars_loc2=self.cars_loc2-rented_cars,
                    )
    
    def return_car(self, location: int, returned_cars: int) -> tuple[Self, float]:
        match location:
            case 1:
                cars_available: int = self.cars_loc1
            case 2:
                cars_available:int = self.cars_loc2
        
        assert 20 >= returned_cars + cars_available, "Cannot have than 20 cars in parking lot"
        
        match location:
            case 1:
                return State(
                    cars_loc1=self.cars_loc1+returned_cars, 
                    cars_loc2=self.cars_loc2,
                    )
            case 2:
                return State(
                    cars_loc1=self.cars_loc1, 
                    cars_loc2=self.cars_loc2+returned_cars,
                    )
    
    def get_all_transitions(self, action: int) -> list[tuple[Self, float, int]]:
        """
        Returns list of (next_state, probability, reward) for all possible transitions
        given an action.
        """
        # 1. Move cars overnight
        next_state, move_reward = self.move_cars(action)
        
        transitions = []
        # 2. Iterate over all possible combinations:
        for rentals_loc1 in range(min(next_state.cars_loc1+1)):
            for returns_loc1 in range(21):
                for rentals_loc2 in range(min(next_state.cars_loc2 + 1)):
                    for returns_loc2 in range(21):
                        if not (returns_loc1 - rentals_loc1 == next_state.cars_loc1 - (self.cars_loc1 - action)):
                            continue
                        elif not (returns_loc2 - rentals_loc2 == next_state.cars_loc2 - (self.cars_loc2 + action)):
                            continue
                        elif not (0 <= rentals_loc1 <= next_state.cars_loc1):
                            continue
                        elif not (0 <= rentals_loc2 <= next_state.cars_loc2):
                            continue
                        elif not (0 <= next_state.cars_loc1 - rentals_loc1 + returns_loc1 <= 20):
                            continue
                        elif not (0 <= next_state.cars_loc2 - rentals_loc2 + returns_loc2 <= 20):
                            continue
                        
                        final_state = State(
                            cars_loc1=next_state.cars_loc1 - rentals_loc1 + returns_loc1,
                            cars_loc2=next_state.cars_loc2 - rentals_loc2 + returns_loc2
                        )
                        
                        # Check if valid state
                        try:
                            final_state.__post_init__()
                        except ValueError:
                            continue
                            
                        # Calculate probabilities
                        rent_loc1_prob: float = self.poisson_prob(rentals_loc1, 3)
                        return_loc1_prob: float = self.poisson_prob(returns_loc1, 3)
                        rent_loc2_prob: float = self.poisson_prob(rentals_loc2, 4)
                        return_loc2_prob: float = self.poisson_prob(returns_loc2, 2)
                        
                        probability: float = rent_loc1_prob * return_loc1_prob * rent_loc2_prob * return_loc2_prob
                        reward: int = move_reward + 10 * (rentals_loc1 + rentals_loc2)
                        
                        transitions.append((final_state, probability, reward))
                        
                        return transitions